# Causal model of urban heat intensity (UHI) using OLS

### Initialization

In [1]:
# import general packages
import numpy as np
import pandas as pd
import warnings
import os
import pickle
import yaml

In [5]:
# ignore warnings & adjust location
warnings.filterwarnings("ignore")
home_directory = os.path.expanduser( '~' )
os.chdir(home_directory + '/DS_Project/modules')

In [6]:
from models.UHI_modeling.disaggregate import *
from models.UHI_modeling.UHI import *

In [7]:
# import stastical equipment
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures

### Data loading and preparation

In [8]:
# load features and target
grid_size_meters = 250
with open(path + 'final_' + str(grid_size_meters) + '_c.pkl', 'rb') as file:
    final = pickle.load(file)
final = final[['id','nLST','wLST','impervious','building','low vegetation','water','trees','road','avg_height']]

In [9]:
# add constant and define features
final = sm.add_constant(final)
features = ['const','building','low vegetation','water','trees','road','avg_height']
features_interact = ['building','low vegetation','water','trees','road']
features_no_interact = ['const','avg_height']
target = "wLST"
final.head()

,const,id,nLST,wLST,impervious,building,low vegetation,water,trees,road,avg_height
0,1.0,1000001,38.415957,38.386701,0.194779,0.273158,0.030052,0.0,0.304353,0.197658,3.951698
1,1.0,1000002,39.372835,39.446180,0.274826,0.318494,0.001854,0.0,0.242964,0.161861,8.014864
2,1.0,1000003,40.157464,40.127583,0.261048,0.344028,0.005544,0.0,0.133975,0.255406,7.031546
3,1.0,1000004,40.600983,40.575132,0.373966,0.322744,0.093476,0.0,0.096984,0.112830,6.401182
4,1.0,1000005,39.881231,39.985541,0.355617,0.297305,0.062737,0.0,0.097315,0.187026,6.416425


In [10]:
X_poly = create_polynomials(final, features_interact, features_no_interact)
X_poly.head()

,building,low vegetation,water,trees,road,building^2,building low vegetation,building water,building trees,building road,...,low vegetation trees,low vegetation road,water^2,water trees,water road,trees^2,trees road,road^2,const,avg_height
0,0.273158,0.030052,0.0,0.304353,0.197658,0.074616,0.008209,0.0,0.083137,0.053992,...,0.009146,0.005940,0.0,0.0,0.0,0.092631,0.060158,0.039069,1.0,3.951698
1,0.318494,0.001854,0.0,0.242964,0.161861,0.101438,0.000591,0.0,0.077383,0.051552,...,0.000451,0.000300,0.0,0.0,0.0,0.059032,0.039326,0.026199,1.0,8.014864
2,0.344028,0.005544,0.0,0.133975,0.255406,0.118355,0.001907,0.0,0.046091,0.087867,...,0.000743,0.001416,0.0,0.0,0.0,0.017949,0.034218,0.065232,1.0,7.031546
3,0.322744,0.093476,0.0,0.096984,0.112830,0.104163,0.030169,0.0,0.031301,0.036415,...,0.009066,0.010547,0.0,0.0,0.0,0.009406,0.010943,0.012731,1.0,6.401182
4,0.297305,0.062737,0.0,0.097315,0.187026,0.088390,0.018652,0.0,0.028932,0.055604,...,0.006105,0.011733,0.0,0.0,0.0,0.009470,0.018200,0.034979,1.0,6.416425


### Modeling

In [11]:
model_init = sm.OLS(final[target], X_poly)
model_fit = model_init.fit()

In [12]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wLST   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.693
Method:                 Least Squares   F-statistic:                     45.45
Date:                Tue, 04 Jul 2023   Prob (F-statistic):           2.41e-91
Time:                        17:28:17   Log-Likelihood:                -667.97
No. Observations:                 414   AIC:                             1380.
Df Residuals:                     392   BIC:                             1469.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
building                   -3.3603      5.930     -0.567      0.571     -15.018       8.298
low vegetation              7.0714      8.085      0.875      0.382      -8.824      22.966
water                      20.3056     16.813      1.208      0.228     -12.750      53.361
trees                     -19.6927      4.513     -4.364      0.000     -28.565     -10.820
road                       -4.9267      8.000     -0.616      0.538     -20.654      10.801
building^2                  5.4299      5.924      0.917      0.360      -6.217      17.077
building low vegetation     6.9432     20.443      0.340      0.734     -33.249      47.135
building water            -57.3228     27.925     -2.053      0.041    -112.225      -2.420
building trees             23.8713      9.116      2.619      0.009       5.949      41.793
building road              -2.0457     13.830     -0.148      0.882     -29.236      25.144
low vegetation^2          -15.3176     10.159     -1.508      0.132     -35.290       4.655
low vegetation water      -48.3851     28.349     -1.707      0.089    -104.120       7.350
low vegetation trees       -1.4721      8.001     -0.184      0.854     -17.202      14.258
low vegetation road        -5.0706     25.530     -0.199      0.843     -55.264      45.123
water^2                   -20.0014     19.335     -1.034      0.302     -58.016      18.013
water trees               -21.4603     17.439     -1.231      0.219     -55.747      12.826
water road                -25.2714     30.590     -0.826      0.409     -85.412      34.869
trees^2                    12.1803      3.713      3.281      0.001       4.881      19.480
trees road                  7.2771     10.953      0.664      0.507     -14.256      28.810
road^2                     10.7787     12.094      0.891      0.373     -12.999      34.557
const                      40.6440      1.478     27.506      0.000      37.739      43.549
avg_height                  0.0587      0.035      1.681      0.094      -0.010       0.127
==============================================================================
Omnibus:                       36.384   Durbin-Watson:                   0.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               79.583
Skew:                          -0.476   Prob(JB):                     5.23e-18
Kurtosis:                       4.926   Cond. No.                     5.29e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.29e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
compute_avg_marginal_effect(model_fit, final, "trees", features_interact, features_no_interact, delta=0.001, step=0.01)

-0.05547094905183051

In [14]:
compute_marginal_effect_at_avg(model_fit, final, "trees", features_interact, features_no_interact, delta=0.001, step=0.01)

-0.055470949051894536

In [15]:
# replace impervious surface by feature
id_example = 1000132
feature = 'trees'
example = final[final.id == id_example][features]
print('Example temperature value: \n', round(final[final.id == id_example][target].item(),2))
print('OLS temperature prediction: \n', round(predict_LST(example, features_interact, features_no_interact, model_fit),2))
example[feature] += 0.05
print('OLS temperature prediction after delta: \n', round(predict_LST(example, features_interact, features_no_interact, model_fit),2))

Example temperature value: 
 36.97
OLS temperature prediction: 
 37.28
OLS temperature prediction after delta: 
 36.86


In [16]:
with open(path_model + 'Causal_Model_' + str(grid_size_meters) + '_a.pkl', 'wb') as file:
    pickle.dump(model_fit, file)